In [74]:
#This code reads the Excel files in the same folder, extracts the commodity futures considered, 
#transforms the data and writes to a single excel file for use in analysis
#Remember to delete Data.xlsx before running code.

#Data is from https://www.cftc.gov/MarketReports/CommitmentsofTraders/index.htm

In [2]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

relevant_keys = ['WHEAT-SRW - CHICAGO BOARD OF TRADE',
                 'CORN - CHICAGO BOARD OF TRADE',
                 'SOYBEANS - CHICAGO BOARD OF TRADE',
                 'SOYBEAN OIL - CHICAGO BOARD OF TRADE',
                 'ROUGH RICE - CHICAGO BOARD OF TRADE',
                 'CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE',
                 'NATURAL GAS - NEW YORK MERCANTILE EXCHANGE',
                 'GOLD - COMMODITY EXCHANGE INC.',
                 'SILVER - COMMODITY EXCHANGE INC.',
                 'SUGAR NO. 11 - ICE FUTURES U.S.']
list_files = []
for file in os.listdir():
    if len(file.split('.xls')) > 1:
        list_files.append(file)
list_files.sort(reverse = True)

data_temp = []
for file in list_files:
    df_temp = pd.DataFrame()
    df = pd.read_excel(file)
    for key in relevant_keys:
        data_temp.append(df.loc[df['Market_and_Exchange_Names'] == key])
    print(f'Finished {file}. Continuing...')
print('Succesfully imported Excel files. Transforming...')

#Clean DataFrames
data_new = []
#Columns we want to keep
select_columns = ['Market_and_Exchange_Names',
                  'Report_Date_as_MM_DD_YYYY',
                  'CFTC_Market_Code',
                  'Open_Interest_All',
                  'NonComm_Positions_Long_All',
                  'NonComm_Positions_Short_All',
                  'NonComm_Postions_Spread_All',
                  'Comm_Positions_Long_All',
                  'Comm_Positions_Short_All',
                  'NonRept_Positions_Long_All',
                  'NonRept_Positions_Short_All']

for df in data_temp:
    #Select only columns we want to keep
    df = df.loc[:, df.columns.isin(select_columns)]
    
    #Generate column of weeknumbers based on date column
    #df['WeekNum'] = df['Report_Date_as_MM_DD_YYYY'].dt.isocalendar().week

    #Calculate Futures Hedging Pressure for 3 agents from column values
    df['HPF_comm'] = (df['Comm_Positions_Short_All']-df['Comm_Positions_Long_All'])/df['Open_Interest_All']
    df['HPF_NonComm'] = (df['NonComm_Positions_Short_All']-df['NonComm_Positions_Long_All'])/df['Open_Interest_All']
    df['HPF_NonRept'] = (df['NonRept_Positions_Short_All']-df['NonRept_Positions_Long_All'])/df['Open_Interest_All']
    #Reorder columns so WeekNum is after date
    cols = df.columns.tolist()
    cols = cols[0:4] + cols[-3:] + cols[4:-3]

    #Set DataFrame to new reordered DataFrame
    df = df[cols]
    data_new.append(df)

data_w, data_c, data_sb, data_sbo, data_rr, data_co, data_ng, data_g, data_si, data_su = [],[],[],[],[],[],[],[],[],[]

for i, df in enumerate(data_new):
    #Reads first entry of dataframe and adds to corresponding list
    if df.iloc[0][0] == relevant_keys[0]:
         data_w.append(df)
    elif df.iloc[0][0] == relevant_keys[1]:
        data_c.append(df)
    elif df.iloc[0][0] == relevant_keys[2]:
        data_sb.append(df)
    elif df.iloc[0][0] == relevant_keys[3]:
        data_sbo.append(df)
    elif df.iloc[0][0] == relevant_keys[4]:
        data_rr.append(df)
    elif df.iloc[0][0] == relevant_keys[5]:
        data_co.append(df)
    elif df.iloc[0][0] == relevant_keys[6]:
        data_ng.append(df)
    elif df.iloc[0][0] == relevant_keys[7]:
        data_g.append(df)
    elif df.iloc[0][0] == relevant_keys[8]:
        data_si.append(df)
    elif df.iloc[0][0] == relevant_keys[9]:
        data_su.append(df)
    else:
        print(f'Error in index of data_new {i}')
#Convert lists of dataframes to dataframes by concatenating
df_w, df_c, df_sb, df_sbo, df_rr, df_co, df_ng, df_g, df_si, df_su = pd.concat(data_w), pd.concat(data_c), pd.concat(data_sb), pd.concat(data_sbo), pd.concat(data_rr), pd.concat(data_co), pd.concat(data_ng), pd.concat(data_g), pd.concat(data_si), pd.concat(data_su)
print('Succesfully transformed data. Writing to Excel...')
filename = 'Data.xlsx'

with pd.ExcelWriter(filename) as writer:
    df_w.to_excel(writer, index = False, header = True, sheet_name = "Wheat")
    df_c.to_excel(writer, index = False, header = True, sheet_name = "Corn")
    df_sb.to_excel(writer, index = False, header = True, sheet_name = "Soybean")
    df_sbo.to_excel(writer, index = False, header = True, sheet_name = "Soybean Oil")
    df_rr.to_excel(writer, index = False, header = True, sheet_name = "Rice")
    df_co.to_excel(writer, index = False, header = True, sheet_name = "Crude Oil")
    df_ng.to_excel(writer, index = False, header = True, sheet_name = "Natural Gas")
    df_g.to_excel(writer, index = False, header = True, sheet_name = "Gold")
    df_si.to_excel(writer, index = False, header = True, sheet_name = "Silver")
    df_su.to_excel(writer, index = False, header = True, sheet_name = "Sugar")

print('Finished.')

Finished 2022.xls. Continuing...
Finished 2021.xls. Continuing...
Finished 2020.xls. Continuing...
Finished 2019.xls. Continuing...
Finished 2018.xls. Continuing...
Finished 2017.xls. Continuing...
Finished 2015-2016.xls. Continuing...
Finished 2007-2014.xls. Continuing...
Succesfully imported Excel files. Transforming...
Succesfully transformed data. Writing to Excel...
Finished.
